In [55]:
from snmp_util import SNMP_runner
from data_struct import Field
import subprocess

In [16]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
%autoreload 2

In [24]:
# test connected
command = "snmpwalk -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 sysUpTime"
token = subprocess.Popen(command)
print('Command success: {}'.format(token.returncode is None))
print(subprocess.check_output(command).decode('utf-8'))

Command success: True
DISMAN-EVENT-MIB::sysUpTimeInstance = Timeticks: (11359823) 1 day, 7:33:18.23



In [49]:
snmp = SNMP_runner()
snmp.walk("sysUpTime")

snmp runtime path: C:\usr\bin
============= init =============

C:\usr\bin>snmpwalk -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 sysUpTime
DISMAN-EVENT-MIB::sysUpTimeInstance = Timeticks: (11447574) 1 day, 7:47:55.74



In [50]:
snmp.set("iswStpMSTRevLevel", idx=0, val=0)
print("============================")
#nmp.test_ShowSetShow("iswStpProtocol", idx=0, val=1)

snmp.test_get_continue_set("iswStpMSTRevLevel", idx=0, val_list=[0, 1, 12345,65535])

#snmp.set("iswStpRegionName", idx=0, val="ABCDEFGHIJKLMNOPQRSTUVWXYZ012345", dtype='str')

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswStpMSTRevLevel.0 = 0
IMP-HF528-MIB::iswStpMSTRevLevel.0 = INTEGER: 0

# get
C:\usr\bin>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswStpMSTRevLevel.0
IMP-HF528-MIB::iswStpMSTRevLevel.0 = INTEGER: 0

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswStpMSTRevLevel.0 = 0
IMP-HF528-MIB::iswStpMSTRevLevel.0 = INTEGER: 0

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswStpMSTRevLevel.0 = 1
IMP-HF528-MIB::iswStpMSTRevLevel.0 = INTEGER: 1

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswStpMSTRevLevel.0 = 12345
IMP-HF528-MIB::iswStpMSTRevLevel.0 = INTEGER: 12345

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswStpMSTRevLevel.0 = 65535
IMP-HF528-MIB::iswStpMSTRevLevel.0 = INTEGER: 65535

# get
C:\usr\bin>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswStpMSTRevLevel.0
IMP-HF528-MIB::iswStpMSTRevLevel.0 = INTEGER: -

In [51]:
snmp.walk("iswStpBridgeForwardDelay")

C:\usr\bin>snmpwalk -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswStpBridgeForwardDelay
IMP-HF528-MIB::iswStpBridgeForwardDelay.0 = INTEGER: 15 second



In [52]:
# val_list=[6, 40]

#print("iswRingProtectRingMode is enable, iswStpBridgeModeSet will not set enable")
#snmp.walk("iswRingProtectRingMode")
print("--------------------")
snmp.walk("iswStpBpduFilter")
snmp.set("iswStpBpduFilter", idx=0, val=1)
snmp.set("iswStpBpduFilter", idx=0, val=2)
# snmp.walk("iswStpBridgeModeSet")



--------------------
C:\usr\bin>snmpwalk -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswStpBpduFilter
IMP-HF528-MIB::iswStpBpduFilter.0 = INTEGER: deny(1)

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswStpBpduFilter.0 = 1
IMP-HF528-MIB::iswStpBpduFilter.0 = INTEGER: deny(1)

# set
C:\usr\bin>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswStpBpduFilter.0 = 2
IMP-HF528-MIB::iswStpBpduFilter.0 = INTEGER: flooding(2)



In [60]:
# set table
set_fields = [
    Field(fname='iswStpPortPriority', val=240, dtype='int'),
    Field(fname='iswStpPortEnable', val=0),  # d 1 enable
    Field(fname='iswStpPortPathCost', val=200000000, dtype='int'),  # 1 .. 200000000, d 20000
    Field(fname='iswStpPortEdgeType', val=1, dtype='int'),  #d 0
]

idx = 28
print("# show")
snmp.walk('iswStpPortTable')

print(f"# set entry {idx}")
snmp.set_table(idx=idx, set_fields=set_fields, row_status=None)

print("# show")
snmp.walk('iswStpPortTable')

# show
C:\usr\bin>snmpwalk -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswStpPortTable
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-1 = INTEGER: 128
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-2 = INTEGER: 128
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-3 = INTEGER: 240
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-4 = INTEGER: 128
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-5 = INTEGER: 128
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-6 = INTEGER: 16
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-7 = INTEGER: 128
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-8 = INTEGER: 128
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-9 = INTEGER: 128
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-10 = INTEGER: 128
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-11 = INTEGER: 128
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-12 = INTEGER: 128
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-13 = INTEGER: 128
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-14 = INTEGER: 128
IMP-HF528-MIB::iswStpPortPriority.iswGEPort-15 = INTEGE

In [67]:
mib_field = {'\
iswAgingTime	OBJECT-TYPE\
		SYNTAX		Integer32 ( 10 .. 600 )\
		UNITS		"second"\
		MAX-ACCESS	read-write'}             

In [72]:
test_form =\
{
    "API":"SG",
    "form_type": "Scale",
}

In [74]:
test_fields = [
    {"name": "iswAgingTime", "read_only": False, "value_list": [], default_value=300},
]

In [75]:
[range(10,600)]

[range(10, 600)]

In [76]:
type(range(10, 600))

range

In [81]:
cmd = walk_command + "iswTpFdbTable"

In [ ]:
raw_return = subprocess.check_output(cmd).decode('utf-8')
format_prefix = r"C:\Users\User>"
print(f"{format_prefix}{cmd}")  # first command
print(f"{raw_return}")  # command output

In [193]:
# 使用情境
# 測試 scaler 
scale('abc').get()
scale('abc').set(299)
scale('abc').get()
"""
# get
C:\Users\User>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswAgingTime.0
IMP-HF528-MIB::iswAgingTime.0 = INTEGER: 300 second
# set
C:\Users\User>snmpset -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswAgingTime.0 = 299
IMP-HF528-MIB::iswAgingTime.0 = INTEGER: 299 second
# get
C:\Users\User>snmpget -v1 -c public -m +IMP-HF528-MIB 172.16.10.8 iswAgingTime.0
IMP-HF528-MIB::iswAgingTime.0 = INTEGER: 299 second
"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 9-10: truncated \UXXXXXXXX escape (3881785731.py, line 16)

In [197]:
#  mib_parser
raw_mib_txt="""
--1
	iswStpProtocol	OBJECT-TYPE
		SYNTAX		Integer32 { stp (0), rstp (1), mstp (2) }
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP protocol specification. Default value is stp (0)."
		DEFVAL		{ 0 }
		::= { iswStp 1 }
--2 
	iswStpPriority	OBJECT-TYPE
		SYNTAX		Integer32 ( 0 | 4096 | 8192 | 12288 | 16384 | 20480 | 24576 | 28672 | 
								32768 | 36864 | 40960 | 45056 | 49152 | 53248 | 57344 | 61440 )
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP priority. Default is 32768(0x8000). Step is 4096."
		DEFVAL		{ 32768 }
		::= { iswStp 2 }
--3
	iswStpTimeSinceTopologyChange	OBJECT-TYPE
		SYNTAX		Gauge32  
		UNITS		"second"
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP time since topology change."
		::= { iswStp 3 } 
--4 
	iswStpTopChanges	OBJECT-TYPE
		SYNTAX		Gauge32
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP top changes."
		::= { iswStp 4 } 
--5 
	iswStpDesignatedRoot	OBJECT-TYPE
		SYNTAX		OCTET STRING ( SIZE (8) )
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP designated root."
		::= { iswStp 5 } 
--6
	iswStpBridgeID	OBJECT-TYPE
		SYNTAX		OCTET STRING ( SIZE (8) )
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP bridge port ID."
		::= { iswStp 6 } 
--7   
	iswStpRootCost	OBJECT-TYPE
		SYNTAX		Integer32
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP root cost."
		::= { iswStp 7 } 
--8
	iswStpRootPort	OBJECT-TYPE
		SYNTAX		Integer32
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP root port. 0 means NA(no root port)."
		::= { iswStp 8 } 
--9   
	iswStpMaxAge	OBJECT-TYPE
		SYNTAX		Integer32
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP max age."
		::= { iswStp 9 } 
--10  
	iswStpHellowTime	OBJECT-TYPE
		SYNTAX		Integer32
		UNITS		"second"
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP hellow time."
		::= { iswStp 10 } 
--11    
	iswStpHoldTime	OBJECT-TYPE
		SYNTAX		Integer32   
		UNITS		"second"
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP hold time."
		::= { iswStp 11 } 
--12
	iswStpForwardDelay	OBJECT-TYPE
		SYNTAX		Integer32
		UNITS		"second"
		MAX-ACCESS	read-only
		STATUS		current
		DESCRIPTION	"STP forward delay."
		::= { iswStp 12 } 
--13    
	iswStpBridgeMaxAge	OBJECT-TYPE
		SYNTAX		Integer32 ( 6 .. 40 ) 
		UNITS		"second"
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP bridge max age. Default value is 20.
					The iswStpBridgeMaxAge, iswStpBridgeHellowTime and iswStpBridgeForwardDelay times are constrained as follows:
					2 x (iswStpBridgeForwardDelay - 1) >= iswStpBridgeMaxAge >= 2 x (iswStpBridgeHellowTime + 1)"
		DEFVAL		{ 20 }
		::= { iswStp 13 } 
--14 
	iswStpBridgeHellowTime	OBJECT-TYPE
		SYNTAX		Integer32 ( 1 .. 10 )
		UNITS		"second"
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP bridge hellow time. Default value is 2.
					The iswStpBridgeMaxAge, iswStpBridgeHellowTime and iswStpBridgeForwardDelay times are constrained as follows:
					2 x (iswStpBridgeForwardDelay - 1) >= iswStpBridgeMaxAge >= 2 x (iswStpBridgeHellowTime + 1)"    
		DEFVAL		{ 2 }
		::= { iswStp 14 } 
--15  
	iswStpBridgeForwardDelay	OBJECT-TYPE
		SYNTAX		Integer32 ( 4 .. 30 )
		UNITS		"second"
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP bridge forward delay. Default value is 15.
					The iswStpBridgeMaxAge, iswStpBridgeHellowTime and iswStpBridgeForwardDelay times are constrained as follows:
					2 x (iswStpBridgeForwardDelay - 1) >= iswStpBridgeMaxAge >= 2 x (iswStpBridgeHellowTime + 1)"
		DEFVAL		{ 15 }
		::= { iswStp 15 } 
--16    
	iswStpBridgeModeSet	OBJECT-TYPE
		SYNTAX		Integer32 { disable (0), enable (1) }
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP bridge mode set. Default value is disable (0).
					If any ring group is enabled (iswRingProtectRingMode is enable), set this field to enable will be rejected."
		DEFVAL		{ 0 }
		::= { iswStp 16 } 
--17    
	iswStpBpduFilter	OBJECT-TYPE
		SYNTAX		Integer32 { deny (1), flooding (2) }
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP bpdu filter. Default value is deny (1)."  
		DEFVAL		{ 1 }
		::= { iswStp 17 } 
--18
	iswStpRegionName	OBJECT-TYPE
		SYNTAX		OCTET STRING ( SIZE ( 0 .. 32 ) )
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"STP Region Name. Default value is emtpy. Max size is 32 chars, preferably in human-readable form."  
		::= { iswStp 18 } 
--19
	iswStpMSTRevLevel	OBJECT-TYPE
		SYNTAX		Integer32 ( 0 .. 65535 )
		MAX-ACCESS	read-write
		STATUS		current
		DESCRIPTION	"MST Revision Level. Default value is 0."  
		DEFVAL		{ 0 }
		::= { iswStp 19 } 
"""

In [207]:
_ = [__.replace('\t',' ').strip() for __ in raw_mib_txt.split('\n')]

In [208]:
for txt in _:
    print(txt)


--1
iswStpProtocol OBJECT-TYPE
SYNTAX  Integer32 { stp (0), rstp (1), mstp (2) }
MAX-ACCESS read-write
STATUS  current
DESCRIPTION "STP protocol specification. Default value is stp (0)."
DEFVAL  { 0 }
::= { iswStp 1 }
--2
iswStpPriority OBJECT-TYPE
SYNTAX  Integer32 ( 0 | 4096 | 8192 | 12288 | 16384 | 20480 | 24576 | 28672 |
32768 | 36864 | 40960 | 45056 | 49152 | 53248 | 57344 | 61440 )
MAX-ACCESS read-write
STATUS  current
DESCRIPTION "STP priority. Default is 32768(0x8000). Step is 4096."
DEFVAL  { 32768 }
::= { iswStp 2 }
--3
iswStpTimeSinceTopologyChange OBJECT-TYPE
SYNTAX  Gauge32
UNITS  "second"
MAX-ACCESS read-only
STATUS  current
DESCRIPTION "STP time since topology change."
::= { iswStp 3 }
--4
iswStpTopChanges OBJECT-TYPE
SYNTAX  Gauge32
MAX-ACCESS read-only
STATUS  current
DESCRIPTION "STP top changes."
::= { iswStp 4 }
--5
iswStpDesignatedRoot OBJECT-TYPE
SYNTAX  OCTET STRING ( SIZE (8) )
MAX-ACCESS read-only
STATUS  current
DESCRIPTION "STP designated root."
::= { iswStp

In [ ]:
#